In [ ]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI

In [ ]:
load_dotenv()
model = ChatMistralAI(
    model="mistral-small-latest",
    temperature=0.1
)

In [ ]:
class RumorSchema(BaseModel):
    topic: str = Field(description="topic(string)")
    cause: str = Field(description="cause(string)")
    source: str = Field(description="source(string)")
    affected_body_part: str = Field(description="affected_body_part(string)")

In [33]:
structured_llm = model.with_structured_output(RumorSchema)

In [ ]:
prompt = ChatPromptTemplate.from_template("""
Given rumor extract the details accoring to given schema given below and return in strict json format
Schema:
topic(string)
cause(string)
source(string)
affected_body_part(string)
If any value isnt present return NAN
Rumor:{rumor}
""")

In [35]:
chain = prompt | structured_llm

In [ ]:
result = chain.invoke({
    "rumor": "from whatsapp, smoking affects stomach and causes cancer"
})
print(result)
print(result.model_dump())
print(type(result))
print(type(result.model_dump()))
#have to return result.model_dump() as dict and convert to json by json.loads(dict)function

topic='smoking affects stomach and causes cancer' cause='smoking' source='whatsapp' affected_body_part='stomach'
{'topic': 'smoking affects stomach and causes cancer', 'cause': 'smoking', 'source': 'whatsapp', 'affected_body_part': 'stomach'}
<class '__main__.RumorSchema'>
<class 'dict'>
